In [ ]:
!pip install --upgrade google-api-python-client

In [ ]:
# Run with Python2
import json, gzip
import sys
import time
import yaml

from random import randint
# pip install --upgrade google-api-python-client
from googleapiclient import discovery

In [ ]:
credentials = yaml.load(open("../credentials.yaml"))["perspective-api"]

In [ ]:
# Generates API client object dynamically based on service name and version. You can request more keys if required in their website
# service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=credentials["service"])

service2 = discovery.build('commentanalyzer', 'v1alpha1', developerKey=credentials["service2"])

description = "This is an example: I hate n*****."
# https://www.perspectiveapi.com/#/
# Expected toxicity (according to the website): 78,75

try:
    if sys.getsizeof(description) > 0 and sys.getsizeof(description) < 3000:
        analyze_request = {
            'comment': { 'text': description },
            'requestedAttributes': {'TOXICITY': {}}
        }
        response = service2.comments().analyze(body=analyze_request).execute()
        toxicityScore = response
    else:
        toxicityScore = None
except:
    toxicityScore = None

print(toxicityScore)